#### Instalacja bibliotek

In [1]:
!pip install pandas feast-azure-provider azureml-core

     |████████████████████████████████| 2.7 MB 4.1 MB/s 
     |████████████████████████████████| 280 kB 88.1 MB/s 
     |████████████████████████████████| 366 kB 65.4 MB/s 
     |████████████████████████████████| 134 kB 51.7 MB/s 
     |████████████████████████████████| 275 kB 57.4 MB/s 
     |████████████████████████████████| 596 kB 66.5 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 50 kB 8.9 MB/s 
     |████████████████████████████████| 11.1 MB 50.9 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 100 kB 12.7 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 2.3 MB 42.3 MB/s 
     |████████████████████████████████| 96 kB 7.1 MB/s 
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 241 kB 84.0 MB/s 
     |████████████████████████████████| 4.0 MB 45.3 MB/s 
     |████████████████████

#### Pobranie kodu z repo

In [1]:
!git clone https://github.com/juskuz/feast-azure-demo-aitech.git

Cloning into 'feast-azure-demo-aitech'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [2]:
!ls

feast-azure-demo-aitech  sample_data


#### Wygenerowanie danych

In [3]:
!ls feast-azure-demo-aitech/data_utils/

generator.py  load_data.py


In [4]:
!python feast-azure-demo-aitech/data_utils/generator.py 

INFO:root:Generating synthetic data from 2022-05-13 to 2022-05-27 for 1000 players...
INFO:root:252750 stats rows generated
INFO:root:34003 payments rows generated
INFO:root:Saving data to /data


#### Stworzenie zasobów Azure

In [5]:
# won't work (az not found)
!az ad signed-in-user show --query objectId -o tsv

/bin/bash: az: command not found


In [7]:
# install Azure CLI for Linux (Debian)
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,765 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.n

In [8]:
!az ad signed-in-user show --query objectId -o tsv

The underlying Active Directory Graph API will be replaced by Microsoft Graph API in Azure CLI 2.37.0. Please carefully review all breaking changes introduced during this migration: https://docs.microsoft.com/cli/azure/microsoft-graph-migration
Please run 'az login' to setup account.


In [ ]:
# logowanie do Azure za pomocą Azure CLI
!az login --allow-no-subscriptions --tenant <paste_here_tenant_id>


In [ ]:
# wyświetl szczegóły dla wybranej subskrypcji
!az account show --subscription <paste_here_subscription_id>

In [11]:
!az ad signed-in-user show --query objectId -o tsv


The underlying Active Directory Graph API will be replaced by Microsoft Graph API in Azure CLI 2.37.0. Please carefully review all breaking changes introduced during this migration: https://docs.microsoft.com/cli/azure/microsoft-graph-migration
__xxxxxxxxxxxxxxxxxxx__


Należy utworzyć zasoby Azure (przez Azure CLI lub Azure Portal). Można wykorzystać template.

#### Załadowanie danych do Synapse (offline store)

Po utworzeniu zasobów należy załadować dane na DataLake (np. używając Synapse), a następnie załadować do bazy (*load_data.py*). Przed uruchomieniem *load_data* należy zaktualizować konfigurację. Zamiast uruchamiać skrypt *load_data.py* można także ręcznie uruchomić zapytania SQL. Skrypt do wykonania z poziomu Synapse/ portalu Azure znajduje się w pliku *load_data.sql*.



Instalacja ODBC: - opcjonalnie, tylko jeśli używamy *load_data.py*.

In [12]:
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list

!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0   7123      0 --:--:-- --:--:-- --:--:--  7071
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    77  100    77    0     0   1013      0 --:--:-- --:--:-- --:--:--  1013
Hit:1 https://packages.microsoft.com/repos/azure-cli bionic InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://packages.microsoft.com/ubuntu/18.04/prod bionic InRelease [4,003 B]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:5 https://packages.microsoft.com/ubuntu/18.04/prod bionic/main amd64 Packages [284 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 ht

Do zaktualizowania konfiguracji możemy użyć pliku *config.json* pobranego z zasobu Azure Machine Learning. Po pobraniu należy go dodać do plików (menu po lewej stronie) oraz sprawdzić poprawność ścieżek w plikach *data_utils/load_data.py* oraz *utils/prepare_feast.py*.

In [19]:
# !python feast-azure-demo-aitech/data_utils/load_data.py
# auth error (401)

W ramach tego demo dane zostaną załadowane ręcznie z poziomu Synaspe. Kolejne kroki wykonywane są podobnie jak we wcześniejszym demo.

In [20]:
# utworzenie repo
!feast init feature_repo

Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage

Creating a new Feast repository in /content/feature_repo.



Po pojawieniu się repo należy usunąć zawartość folderu data w tym repo (feature_repo/data).



W pliku feature_store.yaml należy ustawić konfigurację dla Azure Providera:

In [ ]:
# zawartosć pliku feature_store.yaml

registry:
  registry_store_type: feast_azure_provider.registry_store.AzBlobRegistryStore
  path: ${REGISTRY_PATH}
project: production
provider: feast_azure_provider.azure_provider.AzureProvider
online_store:
    type: redis
    connection_string: ${REDIS_CONN}
offline_store:
    type: feast_azure_provider.mssqlserver.MsSqlServerOfflineStore
    connection_string: ${SQL_CONN}

Po zaktualizowaniu tej konfiguracji należy uruchomić skrypt *prepare_feast.py*. Dla ułatwienia to wykonywane z poziomu innego skryptu (*apply_feast.py*). W pliku opisane są m.in cechy, które mają pojawić się w feature store. Należy też pamiętać o dodaniu pliku z konfiguracją zasobów Azure (plik *config.json* do pobrania np. z Azure Machine Learning Studio)

In [21]:
# !python feast-azure-demo-aitech/utils/prepare_feast.py

/usr/local/lib/python3.7/dist-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
  category=DeprecationWarning)


In [22]:
!python feast-azure-demo-aitech/apply_feast.py

/usr/local/lib/python3.7/dist-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
  category=DeprecationWarning)
05/20/2022 10:25:23 PM INFO:Found the config file in: /content/feast-azure-demo-aitech/config.json
05/20/2022 10:25:28 PM INFO:Created a worker pool for first use


Po ustawieniu cech w feature store można sprawdzić zawartość feature store. Przygotowano w tym celu skrypt *show_stores.py*.

In [23]:
!python feast-azure-demo-aitech/show_stores.py

/usr/local/lib/python3.7/dist-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
  category=DeprecationWarning)
05/20/2022 10:25:48 PM INFO:Found the config file in: /content/feast-azure-demo-aitech/config.json
05/20/2022 10:25:52 PM INFO:Created a worker pool for first use

----------------------HIST DATA FRAME----------------------
SELECT DISTINCT * FROM
(     
    SELECT player_id, ts FROM stats WHERE player_id in ('01C', 'EKS') 
    UNION SELECT player_id, ts FROM payments WHERE player_id in ('01C', 'EKS') 
) res    

Using ts as the event timestamp. To specify a column explicitly, please name it event_timestamp.

----------------------OFFLINE DATA FRAME PLAYER1----------------------
    player_id  ... time_in_game
0         01C  ...  2495.968419
1         01C  ...  2245.121485
3         01C  ...  3227.366002
4         01C  ...   132.986827
5         01C  ...  1868.502434
..        ...  ...        

In [24]:
!python feast-azure-demo-aitech/materialize_stores.py

/usr/local/lib/python3.7/dist-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
  category=DeprecationWarning)
05/20/2022 10:26:52 PM INFO:Found the config file in: /content/feast-azure-demo-aitech/config.json
05/20/2022 10:26:58 PM INFO:Created a worker pool for first use
Materializing 2 feature views from 2022-05-20 21:27:06+00:00 to 2022-05-20 22:27:06+00:00 into the redis online store.

payments:
0it [00:00, ?it/s]
stats:
0it [00:00, ?it/s]


In [25]:
!python feast-azure-demo-aitech/show_stores.py

/usr/local/lib/python3.7/dist-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
  category=DeprecationWarning)
05/20/2022 10:31:03 PM INFO:Found the config file in: /content/feast-azure-demo-aitech/config.json
05/20/2022 10:31:06 PM INFO:Created a worker pool for first use

----------------------HIST DATA FRAME----------------------
SELECT DISTINCT * FROM
(     
    SELECT player_id, ts FROM stats WHERE player_id in ('01C', 'EKS') 
    UNION SELECT player_id, ts FROM payments WHERE player_id in ('01C', 'EKS') 
) res    

Using ts as the event timestamp. To specify a column explicitly, please name it event_timestamp.

----------------------OFFLINE DATA FRAME PLAYER1----------------------
    player_id  ... time_in_game
1         01C  ...  2245.121485
3         01C  ...  1335.816468
5         01C  ...    20.868911
7         01C  ...  1473.485543
8         01C  ...  3338.958819
..        ...  ...        